We re-implement the same solution using CNNs now

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib_inline
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader


In [2]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    # download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    # download=True,
    transform=ToTensor()
)


In [3]:
train_loader = DataLoader(training_data,
                          batch_size=32,
                          shuffle=True
                          )

test_loader = DataLoader(test_data,
                         batch_size=32,
                         shuffle=True)


In [4]:
def op_size(input, kernel, padding, stride):
    return int((((input-kernel+(2*padding))/stride)+1))


In [5]:
class cnn_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.cl1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.cl2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        # op_size(input=8,kernel=3,padding=0,stride=1)*op_size(input=8,kernel=3,padding=0,stride=1)), 100)
        self.fc1 = nn.Linear((16*5*5), 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.cl1(x)))
        x = self.pool(F.relu(self.cl2(x)))
        # op_size(input=8,kernel=3,padding=0,stride=1)*op_size(input=8,kernel=3,padding=0,stride=1)))
        x = x.view(-1, (16*5*5))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x


In [7]:
net = cnn_model()
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.05)
n_total_steps = len(train_loader)

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()     # Does the update

    print(f"Epoch : {epoch+1}. Loss: {loss}")


Epoch : 1. Loss: 0.37823382019996643
Epoch : 2. Loss: 0.44140663743019104
Epoch : 3. Loss: 0.4651886224746704
Epoch : 4. Loss: 0.3191516697406769
Epoch : 5. Loss: 0.24656380712985992


In [9]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for efficiency
    for images, labels in test_loader:
        # images = images.to(device)
        # labels = labels.to(device)

        # Forward pass to get model predictions
        outputs = net(images)

        # Get the index of the predicted class
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")

# Assuming you have already defined your ConvNet model and test_dataloader
# Make sure your model is on the appropriate device (CPU or GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Call the test_model function to evaluate the model on the test set
# test_model(model, test_dataloader, device)


Accuracy on the test set: 87.44%
